**Alunos:**\
*Bruno Berndt Lima - 12542550*\
*Fernando Gonçalves Campos - 12542352*

In [173]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [174]:
# !pip install torcheval
# !pip install caserecommender

In [175]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.decomposition import PCA

import torch
from torch import nn, optim, cuda
from torch.utils.data import TensorDataset, DataLoader

import torcheval
import torcheval.metrics
import torcheval.metrics.regression

from caserec.recommenders.item_recommendation.userknn import UserKNN
from caserec.recommenders.rating_prediction.userknn import UserKNN as UserKNN_ratings

import matplotlib.pyplot as plt

import os

from tqdm.notebook import tqdm

from typing import Callable

In [176]:
# WORKING_DIR: str = '.'
WORKING_DIR: str = '/content/drive/MyDrive/Trabalho Sistemas de Recomendação'

DATASET_DIR: str = f'{WORKING_DIR}/dataset'

RATINGS_FILE: str = f'{DATASET_DIR}/ratings_sample.csv'

EMBEDDINGS_DIR: str = f'{WORKING_DIR}/embeddings'

if not os.path.isdir(EMBEDDINGS_DIR):
    os.mkdir(EMBEDDINGS_DIR)

MODELS_DIR: str = f'{WORKING_DIR}/models'

if not os.path.isdir(MODELS_DIR):
    os.mkdir(MODELS_DIR)

In [177]:
# For similarity calculation
USER_ORIGINAL_RATINGS_WEIGHT: float = 15
USER_RELATIVE_RATINGS_WEIGHT: float = 0

ITEM_ORIGINAL_RATINGS_WEIGHT: float = 15
ITEM_RELATIVE_RATINGS_WEIGHT: float = 0

In [178]:
TEST_SIZE: float = 0.1
TRAIN_TEST_SPLIT_SEED: int = 2

In [179]:
USER_EMBEDDINGS_SIZE: int = 1024 # In the range [1, n_users]
ITEM_EMBEDDINGS_SIZE: int = 400 # In the range [1, n_items]

USER_EMBEDDINGS_NAME: str = f'user_pca{USER_EMBEDDINGS_SIZE}'
ITEM_EMBEDDINGS_NAME: str = f'item_pca{ITEM_EMBEDDINGS_SIZE}'

USER_EMBEDDINGS_FILE: str = f'{EMBEDDINGS_DIR}/{USER_EMBEDDINGS_NAME}.npy'
ITEM_EMBEDDINGS_FILE: str = f'{EMBEDDINGS_DIR}/{ITEM_EMBEDDINGS_NAME}.npy'

FORCE_NEW_EMBEDDINGS: bool = False
SAVE_EMBEDDINGS: bool = True

In [180]:
TRAIN_BATCH_SIZE: int = 256
TEST_BATCH_SIZE: int = 256

In [181]:
N_EPOCHS: int = 30
LEARNING_RATE: float = 0.0005
TOWERS_FINAL_SIZE: int = 512
DROPOUT: float = 0.15

USE_TQDM: bool = True
VERBOSE_TRAINNING: bool = True

MODEL_NAME: str = f'two_tower-{USER_EMBEDDINGS_NAME}-{ITEM_EMBEDDINGS_NAME}'

MODEL_FILE: str = f'{MODELS_DIR}/{MODEL_NAME}.pt'

FORCE_TRAINNING: bool = True
SAVE_MODEL: bool = True

In [182]:
PYTORCH_DEVICE: str = 'cuda' if cuda.is_available() else 'cpu'

print(PYTORCH_DEVICE)

cuda


### Preprocessing

In [183]:
ratings: pd.DataFrame = pd.read_csv(RATINGS_FILE, names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
ratings: pd.DataFrame = ratings[['userId', 'itemId', 'rating']]

map_users: dict[int,int] = {user: idx for idx, user in enumerate(ratings['userId'].unique())}
map_items: dict[int,int] = {item: idx for idx, item in enumerate(ratings['itemId'].unique())}

ratings['userId'] = ratings['userId'].map(map_users)
ratings['itemId'] = ratings['itemId'].map(map_items)

ratings = ratings.sort_values(['userId', 'itemId'])
ratings['rating'] /= 5

existing_users: pd.CategoricalIndex = pd.CategoricalIndex(ratings['userId'].unique())
existing_items: pd.CategoricalIndex = pd.CategoricalIndex(ratings['itemId'].unique())

ratings['userId'] = ratings['userId'].astype('category').cat.set_categories(existing_users)
ratings['itemId'] = ratings['itemId'].astype('category').cat.set_categories(existing_items)

In [184]:
print('ratings')
display(ratings.head())

ratings


,userId,itemId,rating
0,0,0,1.0
1,0,1,0.9
2,0,2,0.8
3,0,3,0.4
4,0,4,1.0


In [185]:
print(f'number of ratings:  {len(ratings)}')
print(f'number of users:  {len(existing_users)}')
print(f'number of items:  {len(existing_items)}')

number of ratings:  190621
number of users:  11090
number of items:  417


In [186]:
train_ratings: pd.DataFrame
test_ratings: pd.DataFrame
train_ratings, test_ratings = train_test_split(ratings, test_size = TEST_SIZE, random_state = TRAIN_TEST_SPLIT_SEED)

### Creating pseudo-embeddings

#### Calculating similarities

In [187]:
# Converts the user ratings into the list format for each user
buffer_df: pd.DataFrame = ratings[['userId', 'itemId', 'rating']].copy()
buffer_df = buffer_df.pivot_table(index='userId', columns='itemId', values='rating', observed=False, fill_value=0)
buffer_df = buffer_df.reindex(index=existing_users.to_numpy(), columns=existing_items.to_numpy(), fill_value=0)

buffer_df = buffer_df.sort_index()

train_user_ratings: np.ndarray = buffer_df.to_numpy()
print(train_user_ratings.shape)

del buffer_df

(11090, 417)


In [188]:
def get_similarity_and_bias(ratings: np.ndarray, original_ratings_weight: float = 1, relative_ratings_weight: float = 1) -> tuple[np.ndarray, np.ndarray]:
    total_weight: float = original_ratings_weight + relative_ratings_weight
    original_ratings_weight: float = 0.5
    relative_ratings_weight: float = 0.5
    if total_weight != 0:
        original_ratings_weight = original_ratings_weight / total_weight
        relative_ratings_weight = relative_ratings_weight / total_weight

    ratings: np.ndarray = ratings.copy()

    relative_ratings: np.ma.MaskedArray = np.ma.masked_array(ratings.copy(), ratings == 0)
    bias: np.ndarray = np.ma.mean(relative_ratings, axis=1).filled(0)
    relative_ratings -= bias[:, np.newaxis]
    relative_ratings += 1

    rating_similarity: np.ndarray = cosine_similarity(ratings, ratings)
    relative_rating_similarity: np.ndarray = cosine_similarity(relative_ratings.filled(0), relative_ratings.filled(0))

    np.fill_diagonal(rating_similarity, 1)
    np.fill_diagonal(relative_rating_similarity, 1)

    similarity = (original_ratings_weight * rating_similarity) + (relative_ratings_weight * relative_rating_similarity)
    np.fill_diagonal(similarity, 1)

    return similarity, bias

In [189]:
user_similarity: np.ndarray
user_bias: np.ndarray
user_similarity, user_bias = get_similarity_and_bias(
    ratings = train_user_ratings,
    original_ratings_weight = USER_ORIGINAL_RATINGS_WEIGHT,
    relative_ratings_weight = USER_RELATIVE_RATINGS_WEIGHT
)

In [190]:
item_similarity: np.ndarray
item_bias: np.ndarray
item_similarity, item_bias = get_similarity_and_bias(
    ratings = train_user_ratings.T,
    original_ratings_weight = ITEM_ORIGINAL_RATINGS_WEIGHT,
    relative_ratings_weight = ITEM_RELATIVE_RATINGS_WEIGHT
)

In [191]:
print(f'User similarity: \n{user_similarity}\n')

User similarity: 
[[1.         0.00907657 0.01073665 ... 0.02045029 0.01639663 0.02832002]
 [0.00907657 1.         0.04230945 ... 0.0194611  0.02884721 0.0182512 ]
 [0.01073665 0.04230945 1.         ... 0.01882644 0.02795121 0.02607122]
 ...
 [0.02045029 0.0194611  0.01882644 ... 1.         0.00634035 0.01221962]
 [0.01639663 0.02884721 0.02795121 ... 0.00634035 1.         0.01916895]
 [0.02832002 0.0182512  0.02607122 ... 0.01221962 0.01916895 1.        ]]



In [192]:
print(f'Item similarity: \n{item_similarity}\n')

Item similarity: 
[[1.         0.00282172 0.00962835 ... 0.         0.         0.        ]
 [0.00282172 1.         0.00746293 ... 0.         0.         0.0031936 ]
 [0.00962835 0.00746293 1.         ... 0.         0.         0.00149265]
 ...
 [0.         0.         0.         ... 1.         0.06666667 0.        ]
 [0.         0.         0.         ... 0.06666667 1.         0.        ]
 [0.         0.0031936  0.00149265 ... 0.         0.         1.        ]]



#### Selecting best users/items

In [193]:
def pca_on_similarity(similarity: np.ndarray, k: int) -> np.ndarray:
    pca: PCA = PCA(n_components=k)
    pca.fit(similarity)
    return pca.transform(similarity)

def pca_embeddings_from_similarity(similarity: np.ndarray, bias: np.ndarray, k: int) -> np.ndarray:
    return np.hstack([pca_on_similarity(similarity, k-1), bias[:,np.newaxis]])

In [194]:
user_embeddings: np.ndarray
item_embeddings: np.ndarray
if FORCE_NEW_EMBEDDINGS or not (os.path.isfile(USER_EMBEDDINGS_FILE) or os.path.isfile(ITEM_EMBEDDINGS_FILE)):
    user_embeddings: np.ndarray = pca_embeddings_from_similarity(user_similarity, user_bias, USER_EMBEDDINGS_SIZE)
    item_embeddings: np.ndarray = pca_embeddings_from_similarity(item_similarity, item_bias, ITEM_EMBEDDINGS_SIZE)

    if SAVE_EMBEDDINGS:
        with open(USER_EMBEDDINGS_FILE, 'wb') as f:
            np.save(f, user_embeddings)
        with open(ITEM_EMBEDDINGS_FILE, 'wb') as f:
            np.save(f, item_embeddings)

else:
    with open(USER_EMBEDDINGS_FILE, 'rb') as f:
        user_embeddings = np.load(f)
    with open(ITEM_EMBEDDINGS_FILE, 'rb') as f:
        item_embeddings = np.load(f)

### Model

#### Preparing the data

In [195]:
def create_tensor_dataset(ratings: pd.DataFrame, user_embeddings: np.ndarray, item_embeddings: np.ndarray) -> TensorDataset:
    users: np.ndarray = ratings['userId'].to_numpy()
    items: np.ndarray = ratings['itemId'].to_numpy()

    input_user_embeddings: np.ndarray = user_embeddings[users,:]
    input_item_embeddings: np.ndarray = item_embeddings[items,:]

    output_ratings: np.ndarray = ratings['rating'].to_numpy()

    input_user_emb_tensor: torch.Tensor = torch.from_numpy(input_user_embeddings).to(dtype=torch.float32)
    input_item_emb_tensor: torch.Tensor = torch.from_numpy(input_item_embeddings).to(dtype=torch.float32)
    output_ratings_tensor: torch.Tensor = torch.from_numpy(output_ratings).to(dtype=torch.float32)

    return TensorDataset(input_user_emb_tensor, input_item_emb_tensor, output_ratings_tensor)

In [196]:
train_loader: DataLoader = DataLoader(
    create_tensor_dataset(train_ratings, user_embeddings, item_embeddings),
    shuffle = True, batch_size = TRAIN_BATCH_SIZE
)

test_loader: DataLoader = DataLoader(
    create_tensor_dataset(test_ratings, user_embeddings, item_embeddings),
    shuffle = False, batch_size = TEST_BATCH_SIZE
)

#### Performance Metrics

In [197]:
class Submetric:
    def __init__(self, submetric_function: Callable, *metrics_used: str):
        self.compute: Callable = submetric_function
        self.metrics_used: list[str] = list(metrics_used)

In [198]:
class MetricsHandler:
    def __init__(self):
        self.metrics: dict[str, torcheval.metrics.Metric[torch.Tensor]] = {}
        self.submetrics: dict[str, Submetric] = {}

    def add_metric(self, metric_name: str, metric: torcheval.metrics.Metric[torch.Tensor]) -> None:
        self.metrics[metric_name] = metric

    def add_submetric(self, submetric_name: str, submetric_function: Callable, *metrics_used: str) -> None:
        self.submetrics[submetric_name] = Submetric(submetric_function, *metrics_used)

    def update(self, input: torch.Tensor, target: torch.Tensor) -> None:
        for metric in self.metrics.values():
            metric.update(input, target)

    def compute(self) -> dict[str, float]:
        computed_metrics: dict[str, float] = {metric_name: metric.compute().item() for metric_name, metric in self.metrics.items()}
        computed_submetrics: dict[str, float] = {submetric_name: submetric.compute(*[computed_metrics[metric_used] for metric_used in submetric.metrics_used]) for submetric_name, submetric in self.submetrics.items()}
        computed_metrics.update(computed_submetrics)
        return computed_metrics

    def reset(self) -> None:
        for metric in self.metrics.values():
            metric.reset()

    def used_metrics(self, include_submetrics: bool = True) -> list[str]:
        used_metrics = list(self.metrics.keys())
        if include_submetrics:
            used_metrics += list(self.submetrics.keys())

        return used_metrics

In [199]:
def evaluate(model: torch.nn.Module, loader: DataLoader, metrics: MetricsHandler) -> dict[str, float]:
    model.eval()

    with torch.no_grad():

        user_emb: torch.Tensor
        item_emb: torch.Tensor
        expected_ratings: torch.Tensor
        for user_emb, item_emb, expected_ratings in loader:
            user_emb = user_emb.to(device=PYTORCH_DEVICE)
            item_emb = item_emb.to(device=PYTORCH_DEVICE)
            target = expected_ratings.to(device=PYTORCH_DEVICE)

            outputs: torch.Tensor = model(user_emb, item_emb)

            # Metrics are measured without the normalization
            metrics.update(outputs*5, target*5)

    computed_metrics: dict[str, float] = metrics.compute()

    metrics.reset()

    return computed_metrics

In [200]:
metrics: MetricsHandler = MetricsHandler()
metrics.add_metric('mse', torcheval.metrics.regression.MeanSquaredError(device = PYTORCH_DEVICE))
metrics.add_metric('r2',  torcheval.metrics.regression.R2Score(device = PYTORCH_DEVICE))
metrics.add_submetric('rmse', lambda mse: np.sqrt(mse), 'mse')

#### Ranking Evaluation

In [201]:
def ranking_precision(recommendations: pd.DataFrame, relevant_items: pd.DataFrame) -> float:
    df: pd.DataFrame = recommendations.copy()
    relevant_items = relevant_items.copy()
    relevant_items['is_relevant'] = 1

    df = df.merge(relevant_items, on=['userId', 'itemId'], how='left')
    df['is_relevant'] = df['is_relevant'].fillna(0)
    df['is_relevant'] = df['is_relevant'].astype(int)

    return df['is_relevant'].mean()

def ranking_recall(recommendations: pd.DataFrame, relevant_items: pd.DataFrame) -> float:
    df: pd.DataFrame = recommendations.copy()
    df['was_recommended'] = 1

    df = df.merge(relevant_items[relevant_items['userId'].isin(df['userId'].unique())], on=['userId', 'itemId'], how='right')
    df['was_recommended'] = df['was_recommended'].fillna(0)
    df['was_recommended'] = df['was_recommended'].astype(int)

    return df['was_recommended'].mean()

def ranking_map(recommendations: pd.DataFrame, relevant_items: pd.DataFrame) -> float:
    df: pd.DataFrame = recommendations.copy()
    relevant_items = relevant_items.copy()
    relevant_items['is_relevant'] = 1

    df = df.merge(relevant_items, on=['userId', 'itemId'], how='left')
    df['is_relevant'] = df['is_relevant'].fillna(0)
    df['is_relevant'] = df['is_relevant'].astype(int)

    n_users: int = len(df['userId'].unique())

    df['numerator'] = df.groupby('userId')['is_relevant'].cumsum()
    df['score'] = df['numerator'] / df['rank']

    df = df[df['is_relevant'] != 0]
    return df.groupby('userId')['score'].mean().sum() / n_users

ranking_metrics: dict[str, Callable] = {"Precision": ranking_precision, "Recall": ranking_recall, "MAP": ranking_map}

In [202]:
def evaluate_ranking(recommendations: pd.DataFrame, relevant_items: pd.DataFrame, k: int | list[int], metrics: dict[str, Callable]) -> dict[str, float]:
    if isinstance(k, int):
        df: pd.DataFrame = recommendations.copy()
        df = df[df['rank'] <= k]

        relevant_items = relevant_items[['userId', 'itemId']].copy()

        return {k: metric(df, relevant_items) for k, metric in metrics.items()}

    else:
        return {f'{key}@{top_k}': value for top_k in k for key, value in evaluate_ranking(recommendations, relevant_items, top_k, metrics).items()}

#### Model

In [203]:
class UserTower(torch.nn.Module):
    def __init__(
        self,
        user_embeddings_size: int,
        output_size: int
    ):
        super(UserTower, self).__init__()

        self.l1 = torch.nn.Linear(user_embeddings_size, 2048, dtype=torch.float32)
        self.drop1 =  torch.nn.Dropout(DROPOUT)
        self.l2 = torch.nn.Linear(2048, 2048, dtype=torch.float32)
        self.drop2 =  torch.nn.Dropout(DROPOUT)
        self.output_l = torch.nn.Linear(2048, output_size, dtype=torch.float32)

    def forward(self, user_embeddings):
        temp = self.drop1(torch.relu(self.l1(user_embeddings)))
        temp = self.drop2(torch.relu(self.l2(temp)))
        return torch.relu(self.output_l(temp))

In [204]:
class ItemTower(torch.nn.Module):
    def __init__(
        self,
        item_embeddings_size: int,
        output_size: int
    ):
        super(ItemTower, self).__init__()

        self.l1 = torch.nn.Linear(item_embeddings_size, 2048, dtype=torch.float32)
        self.drop1 =  torch.nn.Dropout(DROPOUT)
        self.l2 = torch.nn.Linear(2048, 2048, dtype=torch.float32)
        self.drop2 =  torch.nn.Dropout(DROPOUT)
        self.output_l = torch.nn.Linear(2048, output_size, dtype=torch.float32)

    def forward(self, item_embeddings):
        temp = self.drop1(torch.relu(self.l1(item_embeddings)))
        temp = self.drop2(torch.relu(self.l2(temp)))
        return torch.relu(self.output_l(temp))

In [205]:
class TwoTowerModel(torch.nn.Module):
    def __init__(
        self,
        user_embeddings_size: int,
        item_embeddings_size: int,
        towers_final_size: int,
    ):
        super(TwoTowerModel, self).__init__()

        self.user_tower: UserTower = UserTower(user_embeddings_size, towers_final_size)
        self.item_tower: ItemTower = ItemTower(item_embeddings_size, towers_final_size)

    def forward(
        self,
        user_embeddings,
        item_embeddings
    ):
        user_temp: torch.Tensor = self.user_tower(user_embeddings)
        item_temp: torch.Tensor = self.item_tower(item_embeddings)

        return torch.sum(user_temp * item_temp, dim=1)

#### Training

In [206]:
model: torch.nn.Module
loss_fn: torch.nn.MSELoss = torch.nn.MSELoss(reduction='mean')

if FORCE_TRAINNING or not os.path.isfile(MODEL_FILE):
    model = TwoTowerModel(
        user_embeddings_size = USER_EMBEDDINGS_SIZE,
        item_embeddings_size = ITEM_EMBEDDINGS_SIZE,
        towers_final_size =    TOWERS_FINAL_SIZE
    )
    model.to(device=PYTORCH_DEVICE)

    optimizer: torch.optim.Adam = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

    try:
        for epoch in (tqdm(range(1, N_EPOCHS+1), desc='Training', unit='epoch') if USE_TQDM else range(1, N_EPOCHS+1)):
            if VERBOSE_TRAINNING or not USE_TQDM:
                print(f'current epoch: {epoch} / {N_EPOCHS}', end=('\n' if VERBOSE_TRAINNING else '\r'))

            model.train()

            epoch_loss = 0

            user_emb: torch.Tensor
            item_emb: torch.Tensor
            expected_ratings: torch.Tensor
            for user_emb, item_emb, expected_ratings in train_loader:
                optimizer.zero_grad()

                user_emb = user_emb.to(device=PYTORCH_DEVICE)
                item_emb = item_emb.to(device=PYTORCH_DEVICE)
                expected_ratings = expected_ratings.to(device=PYTORCH_DEVICE)

                outputs: torch.Tensor = model(user_emb, item_emb)
                loss: torch.Tensor = torch.sqrt(loss_fn(outputs, expected_ratings))

                loss.backward()
                optimizer.step()

                epoch_loss += np.square(loss.item()) * len(user_emb)

            if VERBOSE_TRAINNING:
                # RMSE assuming normalized input
                tqdm.write(f'epoch loss: {np.sqrt((epoch_loss * 25) / len(train_loader.dataset))}')
                tqdm.write(f'test loss: {evaluate(model, test_loader, metrics)["rmse"]}\n')

    except KeyboardInterrupt:
        pass

    if not (VERBOSE_TRAINNING or USE_TQDM):
        print('')

    if SAVE_MODEL:
        torch.save(model, MODEL_FILE)

else:
    model = torch.load(MODEL_FILE)
    model.to(device = PYTORCH_DEVICE)

Training:   0%|          | 0/30 [00:00<?, ?epoch/s]

current epoch: 1 / 30
epoch loss: 0.8636272120394067
test loss: 0.7996720610451619

current epoch: 2 / 30
epoch loss: 0.8077224456347961
test loss: 0.8072354615037319

current epoch: 3 / 30
epoch loss: 0.782554345736983
test loss: 0.7736345290637119

current epoch: 4 / 30
epoch loss: 0.7523125982216657
test loss: 0.7628398372466509

current epoch: 5 / 30
epoch loss: 0.7349074475368147
test loss: 0.750137594635049

current epoch: 6 / 30
epoch loss: 0.7185213377794416
test loss: 0.7215534765737492

current epoch: 7 / 30
epoch loss: 0.7069904945554767
test loss: 0.7209412716294386

current epoch: 8 / 30
epoch loss: 0.6934416324031702
test loss: 0.7173299444353727

current epoch: 9 / 30
epoch loss: 0.680866769392423
test loss: 0.7294184023145226

current epoch: 10 / 30
epoch loss: 0.6634382942315667
test loss: 0.6943671196697272

current epoch: 11 / 30
epoch loss: 0.6474015335625665
test loss: 0.6988459824167045

current epoch: 12 / 30
epoch loss: 0.6363389222842324
test loss: 0.6949526251

### Validation

In [207]:
PRECISION: int = 5
MAX_METRIC_NAME_LENGTH: int = len(max(metrics.used_metrics(), key=len))

computed_train_eval: dict[str, float] = evaluate(model, train_loader, metrics)
print('Train Eval:')
print('\n'.join(f'{metric_name:<{MAX_METRIC_NAME_LENGTH}} : {metric_eval:.{PRECISION}f}' for metric_name, metric_eval in computed_train_eval.items()))

computed_test_eval: dict[str, float] = evaluate(model, test_loader, metrics)
print('\nTest Eval:')
print('\n'.join(f'{metric_name:<{MAX_METRIC_NAME_LENGTH}} : {metric_eval:.{PRECISION}f}' for metric_name, metric_eval in computed_test_eval.items()))

Train Eval:
mse  : 0.17051
r2   : 0.84118
rmse : 0.41293

Test Eval:
mse  : 0.43297
r2   : 0.60104
rmse : 0.65800


In [208]:
def get_predictions(model: torch.nn.Module, df: pd.DataFrame, user_embeddings: np.ndarray, item_embeddings: np.ndarray) -> pd.DataFrame:
    predictions: pd.DataFrame = pd.DataFrame({"userId": [], "itemId": [], "prediction": []})
    predictions['userId'] = predictions['userId'].astype(int)
    predictions['itemId'] = predictions['itemId'].astype(int)

    items_set: set[int] = set(df['itemId'])
    model.eval()

    with torch.no_grad():
        for user in tqdm(df['userId'].unique()):
            user_df: pd.DataFrame = df[df['userId'] == user]
            new_items_ids: list[int] = list(items_set - set(user_df['itemId']))

            input_user_embeddings: torch.Tensor = torch.from_numpy(np.repeat(user_embeddings[user][np.newaxis, :], len(new_items_ids), axis=0)).to(device=PYTORCH_DEVICE, dtype=torch.float32)
            input_item_embeddings: torch.Tensor = torch.from_numpy(item_embeddings[new_items_ids,:]).to(device=PYTORCH_DEVICE, dtype=torch.float32)

            user_predictions: list[float] = model(input_user_embeddings, input_item_embeddings).squeeze().tolist()

            predictions = pd.concat([predictions, pd.DataFrame({"userId": user, "itemId": new_items_ids, "prediction": user_predictions})])

    predictions['rank'] = predictions.groupby("userId")['prediction'].rank('first', ascending=False)
    predictions['rank'] = predictions['rank'].astype(int)
    predictions = predictions.sort_values(["userId", "rank"])
    return predictions

ranked_predictions: pd.DataFrame = get_predictions(model, train_ratings, user_embeddings, item_embeddings)
display(ranked_predictions.head())

  0%|          | 0/11090 [00:00<?, ?it/s]

,userId,itemId,prediction,rank
355,0,367,1.091349,1
0,0,0,1.052030,2
143,0,154,1.048440,3
1,0,5,1.003949,4
96,0,107,1.000226,5


In [209]:
evaluate_ranking(ranked_predictions, test_ratings, k = [5, 10, 20], metrics = ranking_metrics)

{'Precision@5': 0.05424706943192065,
 'Recall@5': 0.15779258248963962,
 'MAP@5': 0.1660705340146278,
 'Precision@10': 0.03630297565374211,
 'Recall@10': 0.21119446047316792,
 'MAP@10': 0.16956840258395714,
 'Precision@20': 0.02391794409377818,
 'Recall@20': 0.27828778261553794,
 'MAP@20': 0.16780108203738203}

### User KNN for comparison

In [210]:
TRAIN_RATINGS_FILE: str = f'train.csv'
TEST_RATINGS_FILE: str = f'test.csv'
USERKNN_OUT_FILE: str = f'{WORKING_DIR}/userknn_out.csv'

KNN for rating prediction

In [218]:
# Going back to the original ratings
train_ratings['rating'] *= 5
test_ratings['rating'] *= 5
train_ratings.to_csv(TRAIN_RATINGS_FILE, index=False, header=False, sep='\t')
test_ratings.to_csv(TEST_RATINGS_FILE, index=False, header=False, sep='\t')
train_ratings['rating'] /= 5
test_ratings['rating'] /= 5

UserKNN_ratings(TRAIN_RATINGS_FILE, TEST_RATINGS_FILE).compute()

[Case Recommender: Rating Prediction > UserKNN Algorithm]

train data:: 11090 users and 410 items (171558 interactions) | sparsity:: 96.23%
test data:: 8704 users and 300 items (19063 interactions) | sparsity:: 99.27%

training_time:: 4.968095 sec
prediction_time:: 81.110458 sec
Eval:: MAE: 0.643044 RMSE: 0.834189 


KNN for item recommendation

In [212]:
train_ratings.to_csv(TRAIN_RATINGS_FILE, index=False, header=False, sep='\t')
test_ratings.to_csv(TEST_RATINGS_FILE, index=False, header=False, sep='\t')
if not os.path.isfile(USERKNN_OUT_FILE):
    UserKNN(TRAIN_RATINGS_FILE, TEST_RATINGS_FILE, output_file=USERKNN_OUT_FILE, as_similar_first=False, rank_length=20).compute() # Creates the predictions for the any pair that didn't appear on the train data

In [213]:
userknn_predictions: pd.DataFrame = pd.read_csv(USERKNN_OUT_FILE, names=['userId', 'itemId', 'prediction'], sep='\t', header=0)

userknn_predictions['rank'] = userknn_predictions.groupby("userId")['prediction'].rank('first', ascending=False)
userknn_predictions['rank'] = userknn_predictions['rank'].astype(int)
userknn_predictions = userknn_predictions.sort_values(["userId", "rank"])

display(userknn_predictions.head())

,userId,itemId,prediction,rank
0,0,5,48.658928,1
1,0,22,48.387157,2
2,0,30,48.238627,3
3,0,57,48.199488,4
4,0,17,48.058980,5


In [214]:
display(userknn_predictions)

,userId,itemId,prediction,rank
0,0,5,48.658928,1
1,0,22,48.387157,2
2,0,30,48.238627,3
3,0,57,48.199488,4
4,0,17,48.058980,5
...,...,...,...,...
221794,11089,30,49.853099,16
221795,11089,18,49.739881,17
221796,11089,33,49.474116,18
221797,11089,13,49.285410,19


In [215]:
evaluate_ranking(userknn_predictions, test_ratings, k = [5, 10, 20], metrics = ranking_metrics)

{'Precision@5': 0.1265825067628494,
 'Recall@5': 0.36820017835597757,
 'MAP@5': 0.3023722572888488,
 'Precision@10': 0.09131650135256988,
 'Recall@10': 0.5312385248911504,
 'MAP@10': 0.29840413690774376,
 'Precision@20': 0.06053228373437211,
 'Recall@20': 0.7042962807532918,
 'MAP@20': 0.28347560988803333}

### Comparison

In [216]:
# Evaluation considering only items that each user didn't dislike as being relevant
ranking_eval = evaluate_ranking(ranked_predictions, test_ratings[test_ratings['rating'] >= 0.6], k = [5, 10, 20], metrics = ranking_metrics)
knn_eval = evaluate_ranking(userknn_predictions, test_ratings[test_ratings['rating'] >= 0.6], k = [5, 10, 20], metrics = ranking_metrics)

In [217]:
display(pd.DataFrame({'model': ranking_eval, 'knn': knn_eval}))

,model,knn
Precision@5,0.053562,0.108711
Recall@5,0.197002,0.399841
MAP@5,0.164973,0.273853
Precision@10,0.035591,0.076023
Recall@10,0.261807,0.559233
MAP@10,0.168451,0.272216
Precision@20,0.023260,0.049581
Recall@20,0.342200,0.729438
MAP@20,0.167003,0.261846
